## Inteligencia Artificial
## - Laboratorio 5 -
Autores:
- Mark Albrand 
- Jimena Hernández

### Task 1 - Graph-Search

In [3]:
import matplotlib.pylab as plt
import numpy as np

### Task 1.1 - Discretización de la imagen

In [23]:
from PIL import Image
import numpy as np
def convertir_a_discreto(imagen_path, tamano_bloque):
    # Cargar la imagen
    imagen = Image.open(imagen_path)
    imagen_np = np.array(imagen)
    
    # Dimensiones de la imagen
    alto, ancho, _ = imagen_np.shape
    
    # Crear una imagen de salida
    imagen_salida = np.zeros_like(imagen_np)
    
    for y in range(0, alto, tamano_bloque):
        for x in range(0, ancho, tamano_bloque):
            # Extraer el bloque
            bloque = imagen_np[y:y+tamano_bloque, x:x+tamano_bloque]
            
            # Calcular el color promedio del bloque
            color_promedio = bloque.mean(axis=(0,1)).astype(int)
            #convertir los colores difuminados al color mas predominante
            #verde
            if color_promedio[1] > color_promedio[0] and color_promedio[1] > color_promedio[2]:
                color_promedio[1] = 255
                color_promedio[0] = 0
                color_promedio[2] = 0 
            #rojo
            elif color_promedio[0] > color_promedio[1] and color_promedio[0] > color_promedio[2]:
                color_promedio[0] = 255
                color_promedio[1] = 0
                color_promedio[2] = 0

            #grises
            elif color_promedio[0] == color_promedio[1] and color_promedio[0] == color_promedio[2] and color_promedio[0]< 255:
                color_promedio[0] = 0
                color_promedio[1] = 0
                color_promedio[2] = 0
            
            # Asignar el color promedio al bloque correspondiente en la imagen de salida
            imagen_salida[y:y+tamano_bloque, x:x+tamano_bloque] = color_promedio
    
    imagen_discreta = Image.fromarray(imagen_salida)
    return imagen_discreta

ruta_imagen = 'images/1.bmp' 
tamano_bloque = 14 #bloques de 14x14
imagen_discreta = convertir_a_discreto(ruta_imagen, tamano_bloque)
imagen_discreta.show() # Mostrar la imagen


Las áreas blancas representan caminos libres
Las áreas negras representan paredes sobre las cuales no se puede pasar
Las áreas verdes representan la meta (goalTest positivos) (pueden ser varios)
La área roja representa el punto de inicio (solo podrá haber uno)

In [6]:
print(type(imagen_discreta))

<class 'PIL.Image.Image'>


In [22]:
# Pasar la imagen a un arreglo de numpy
imagen_np = np.array(imagen_discreta)
print(imagen_np.shape)

height, width, channels = imagen_np.shape

(582, 582, 3)
3


### Task 1.2 - Framework de Problemas

In [ ]:
#Clase actions(s), stepCost(s,a,s’), etcétera)
''''
Puede entenderse como un framework con los siguientes
elementos
○ Estado inicial (s_start)
○ Función de acción actions(s) → {a 1, a 2,...,a n}
○ Función de costo de paso cost(s,a,s’) → R
○ Función de resultados results(s,a) → s’
○ Función de goalTest(s) → {True, False}
● Además, algunas veces es útil agregar
○ Función de costo de ruta pathCost(s 1, s 2,...,s n) → R
■ s i = results(s i-1,a i )
'''
class SolveMaze:
    maze: np.array
    starting_point: tuple[int, int]  # (y, x)
    actions = {
        "up": (-1, 0),
        "down": (1, 0),
        "left": (0, -1),
        "right": (0, 1)
    }

    def __init__(self, maze: np.array):
        if maze.ndim != 3:
            raise ValueError("Maze debe ser un arreglo tridimensional")

        self.maze = maze
        
